In [2]:
import os
from ossapi import Ossapi
import sqlite3
from time import strftime, localtime
import time
from concurrent.futures import ThreadPoolExecutor
import sys
from numpy import array_split
import pickle
from datetime import datetime
sys.path.insert(0, "../") # Required for database in different directory. 

OSU_CLIENT_ID = os.environ.get('OSU_CLIENT_ID')
OSU_CLIENT_SECRET = os.environ.get('OSU_CLIENT_SECRET')

api = Ossapi(OSU_CLIENT_ID, OSU_CLIENT_SECRET)

with open("leaderboard_ids.pickle", "rb") as handle:
    leaderboard_ids = pickle.load(handle)
leaderboard_ids = set(leaderboard_ids)

top_or_recent = "recent"
mode = "std"


In [3]:
num_done = 0
last_time = time.time()

def scrape_users(ids):
    """
    Adds user data to users table and scores to scores in ../data/osu.db after scraping.
    ids: list of ids to scrape
    """
    global num_done
    global last_time
    conn = sqlite3.connect("../data/UserScores.db")  # Change to osu.db in the future
    cursor = conn.cursor()
    try:
        for user_id in ids:
            try:
                top_scores = api.user_scores(user_id, type="recent", mode="osu", limit=100)
                scores = []
            except Exception as e:
                print(e)
                continue

            for score in top_scores:
                try:
                    beatmap = getattr(score, "beatmap", None)
                    beatmap_id = getattr(beatmap, "id", None) if beatmap else None
                    
                    beatmapset_id = getattr(beatmap, "beatmapset_id", None) if beatmap else None

                    mods = getattr(score, "mods", None)
                    mods = getattr(mods, "value", None) if mods else None

                    pp = getattr(score, "pp", None)

                    created_at = getattr(score, "created_at", None)
                    created_at = (
                        datetime.strftime(created_at, "%Y-%m-%d %H:%M:%S")
                        if created_at
                        else None
                    )

                    score = (
                        int(user_id),
                        beatmap_id,
                        beatmapset_id,
                        pp,
                        mods,
                        created_at,
                    )
                    scores.append(score)

                except Exception as e:
                    print(e)
                    continue

            try:
                cursor.executemany(
                    f"""
                    INSERT INTO {top_or_recent}_scores_{mode} (user_id, beatmap_id, beatmapset_id, pp, mods, created_at)
                    VALUES (?, ?, ?, ?, ?, ?)
                    """,
                    scores,
                )
                conn.commit()
            except Exception as e:
                print(e)
                continue

            num_done += 1
            if num_done % 100 == 0:
                time.sleep(16)
                print(
                    str(num_done) + ": " + str(time.time() - last_time),
                    strftime("%H:%M:%S", localtime(time.time())),
                )
                last_time = time.time()
                
    except Exception as e:
        print(e)
    finally:
        conn.close()

In [4]:
conn = sqlite3.connect('../data/UserScores.db')
cursor = conn.cursor()


completed_user_ids = cursor.execute(f"SELECT DISTINCT user_id FROM {top_or_recent}_scores_{mode}").fetchall()
completed_user_ids = set([x[0] for x in completed_user_ids])
conn.close()

remaining_ids = list(leaderboard_ids - completed_user_ids)
print(f"Remaining user ids to scrape: {len(remaining_ids)}")

num_partitions = 5
partitioned_user_ids = array_split(remaining_ids, num_partitions)

for user_ids in partitioned_user_ids:
    print(len(user_ids))

with ThreadPoolExecutor(max_workers=num_partitions) as executor:
    for user_ids in partitioned_user_ids:
        executor.submit(scrape_users, user_ids)
        


Remaining user ids to scrape: 665020
133004
133004
133004
133004
133004
api returned an error of `None` for a request to https://osu.ppy.sh/api/v2/users/7521530/scores/recent?mode=osu&limit=100
api returned an error of `None` for a request to https://osu.ppy.sh/api/v2/users/15910145/scores/recent?mode=osu&limit=100
api returned an error of `None` for a request to https://osu.ppy.sh/api/v2/users/25982752/scores/recent?mode=osu&limit=100
api returned an error of `None` for a request to https://osu.ppy.sh/api/v2/users/10894949/scores/recent?mode=osu&limit=100
100: 4.840483903884888 18:13:47
api returned an error of `None` for a request to https://osu.ppy.sh/api/v2/users/11302740/scores/recent?mode=osu&limit=100
api returned an error of `None` for a request to https://osu.ppy.sh/api/v2/users/17594198/scores/recent?mode=osu&limit=100
api returned an error of `None` for a request to https://osu.ppy.sh/api/v2/users/20104537/scores/recent?mode=osu&limit=100
api returned an error of `None` for 

KeyboardInterrupt: 

api returned an error of `Too Many Attempts.` for a request to https://osu.ppy.sh/api/v2/users/19724016/scores/recent?mode=osu&limit=100


api returned an error of `Too Many Attempts.` for a request to https://osu.ppy.sh/api/v2/users/13842988/scores/recent?mode=osu&limit=100
api returned an error of `Too Many Attempts.` for a request to https://osu.ppy.sh/api/v2/users/21407725/scores/recent?mode=osu&limit=100
api returned an error of `Too Many Attempts.` for a request to https://osu.ppy.sh/api/v2/users/34404087/scores/recent?mode=osu&limit=100
api returned an error of `Too Many Attempts.` for a request to https://osu.ppy.sh/api/v2/users/12180916/scores/recent?mode=osu&limit=100
api returned an error of `Too Many Attempts.` for a request to https://osu.ppy.sh/api/v2/users/15940141/scores/recent?mode=osu&limit=100
api returned an error of `Too Many Attempts.` for a request to https://osu.ppy.sh/api/v2/users/15529720/scores/recent?mode=osu&limit=100
api returned an error of `Too Many Attempts.` for a request to https://osu.ppy.sh/api/v2/users/26860981/scores/recent?mode=osu&limit=100
api returned an error of `Too Many Attemp

In [ ]:
conn.close()

CREATE TABLE IF NOT EXISTS top_scores_std (
    user_id INTEGER,
    beatmap_id INTEGER,
    beatmapset_id INTEGER,
    pp INTEGER,
    mods INTEGER,
    created_at TEXT
);